# Web Crawling con Python


El objetivo de este trabajo práctico es implementar un web crawler en Python que pueda
recorrer un sitio web, extraer todas las etiquetas <a> con sus respectivos enlaces y acceder a
cada página enlazada. Por cada enlace encontrado, se deben obtener todas las etiquetas <h1> y
<p> y almacenarlo en un array en un archivo JSON y si no se encuentran dichos elementos
guardar el array como vacío. Por ejemplo, si el crawler encuentra un enlace
https://example.com/pagina1 en una página y accede a ese enlace, debe obtener el contenido
de la página https://example.com/pagina1 y buscar todos los elementos <h1></h1> y
<p></p>, luego almacenarlo en un array en un archivo JSON bajo la clave
"https://example.com/pagina1" con el array de los elementos solicitados de la página como
valor correspondiente y si no se encuentran dichos elementos guardar el array vacío.


1. Importamos las librerias 

In [ ]:
import requests
from bs4 import BeautifulSoup
import json

2. Creamos una funcion para extraer los datos de una pagina web

In [ ]:
def crawl_website(url): 
    try:
        response = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada
        response.raise_for_status()  # Verifica si hay errores HTTP en la respuesta
        soup = BeautifulSoup(response.text, 'html.parser')  # Crea un objeto BeautifulSoup para analizar el contenido HTML
        
        links = soup.find_all('a', href=True)[:100]  # Encuentra todas las etiquetas <a> con atributo href y obtiene los primeros 100 enlaces
                                                     # Utilice [:100] para limitar la búsqueda a los primeros 100 enlaces para evitar sobrecarga de recursos
                                                     # Para obtener todos los enlaces, elimine [:100] o comentelo

        data = {}  # Inicializa un diccionario para almacenar los datos extraídos de las páginas enlazadas
        
        for link in links:  # Itera sobre cada enlace encontrado
            href = link['href']  # Obtiene la URL del enlace
            if href.startswith('http'):  # Verifica si el enlace comienza con 'http' (enlace externo)
                try:
                    page_response = requests.get(href)  # Realiza una solicitud GET al enlace
                    page_response.raise_for_status()  # Verifica si hay errores HTTP en la respuesta
                    page_soup = BeautifulSoup(page_response.text, 'html.parser')  # Crea un objeto BeautifulSoup para analizar el contenido HTML de la página enlazada
                    h1_tags = page_soup.find_all('h1')  # Encuentra todas las etiquetas <h1> en la página enlazada
                    p_tags = page_soup.find_all('p')  # Encuentra todas las etiquetas <p> en la página enlazada
                    if h1_tags or p_tags:  # Verifica si hay etiquetas <h1> o <p> en la página enlazada
                        data[href] = [str(tag) for tag in h1_tags + p_tags]  # Almacena las etiquetas encontradas como una lista de cadenas en el diccionario 'data'
                    else:
                        data[href] = []  # Si no se encuentran etiquetas <h1> ni <p>, almacena una lista vacía en el diccionario 'data'
                except requests.exceptions.RequestException as e:  # Maneja las excepciones relacionadas con las solicitudes HTTP
                    print(f"Error al acceder a {href}: {e}")  # Imprime un mensaje de error
                except Exception as e:  # Maneja cualquier otro tipo de excepción
                    print(f"Error inesperado al procesar {href}: {e}")  # Imprime un mensaje de error
                    
        return data  # Retorna el diccionario 'data' con los datos extraídos de las páginas enlazadas
    
    except requests.exceptions.RequestException as e:  # Maneja las excepciones relacionadas con las solicitudes HTTP
        print("Error al realizar la solicitud:", e)  # Imprime un mensaje de error
        return {}  # Retorna un diccionario vacío si ocurre un error en la solicitud HTTP